The following is for a daytime model. For a nightime model, remove R12 (as there is no O available to oxidize water to OH) and include R13, R14, and R18.

In [1]:
"""A zero-dimensional photochemical model. Adapted from MATLAB code by 
R. Wordsworth 3/8/20."""    

# define system constants
S2DAY = 1/(3600*24) # 1 day [s]

# Import the necessary packages and modules
import numpy as np
from numpy import exp
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
    
# define all classes

class ChemSpecies:
    """Contains definitions for each chemical species"""    
    def __init__(self,index,name,elements,molmass,behaviour):
        self.index       = index # index number
        self.name        = name # name
        self.elements    = elements # elements tally (integer array)
        self.molmass     = molmass # molar mass [g/mol] (real)
        self.behaviour   = behaviour # 'active' or 'passive' (character)


class Reaction:
    """Contains definitions for each reaction"""    
    def __init__(self,formula):
        self.formula   = formula # reaction formula
        self.reactants = formula.split('=')[0].split('+') # reactants
        self.products  = formula.split('=')[1].split('+') # products


class PhotoReaction(Reaction):
    """Contains definitions for each photoreaction"""    
    def __init__(self,formula,J):
        Reaction.__init__(self,formula)
        self.rateconst = J # reaction rate constant [1/s]
        
        
class TwoBodyReaction(Reaction):
    """Contains definitions for each two-body reaction"""    
    def __init__(self,formula,k_fn):
        Reaction.__init__(self,formula)
        self.k_fn=k_fn
    def calc_rate(self,T):
        self.rateconst = self.k_fn(T) # reac. rate constant [1/molec./cm3]
        
        
class ThreeBodyReaction(Reaction):
    """Contains definitions for each three-body reaction"""    
    def __init__(self,formula,k_fn):
        Reaction.__init__(self,formula)
        self.k_fn=k_fn
    def calc_rate(self,T,psi_tot):
        self.rateconst = self.k_fn(T)*psi_tot # reac. rate constant [1/molec./cm3]


class ReactionNetwork:
    """
    Contains definitions for an entire reaction network:
    chemical reaction network matrices A, q.
    In a pure chemical system, (psi_i)_t = A_ij*psi_j + q_ijk*psi_j*psi_k,
    where psi is the concentration in molecules/cm3 of each species.
    A is for unimolecular reactions, q is for 2- and 3-body reactions
    """    
    def __init__(self,n_species):
        self.A = np.zeros((n_species,n_species))
        self.q = np.zeros((n_species,n_species,n_species))

    def add_reactions(self,reactions,species_dict):
        for i in range(1,len(reactions)+1):
            self.add_reaction(reactions['R'+str(i)],species_dict)

    def add_reaction(self,reaction,species_dict):
        active = lambda species: species.behaviour=='active'
        # only 'active' species have their concentrations updated
        r1 = species_dict[reaction.reactants[0]] # 1st reactant
        p1 = species_dict[reaction.products[0]] # 1st product   
        # the 'ir1' etc. variables are simply here to make the code cleaner
        ir1 = r1.index
        ip1 = p1.index
        
        if(isinstance(reaction,PhotoReaction)): # by default 1 reactant, 2 products
            p2 = species_dict[reaction.products[1]] # 2nd product
            ip2 = p2.index
            self.A[ir1,ir1] -= active(r1)*reaction.rateconst
            self.A[ip1,ir1] += active(p1)*reaction.rateconst
            self.A[ip2,ir1] += active(p2)*reaction.rateconst
            
        elif(isinstance(reaction,TwoBodyReaction)): # by default 2 reactants, 2 products
            r2 = species_dict[reaction.reactants[1]] # 2nd reactant
            p2 = species_dict[reaction.products[1]] # 2nd product
            ir2 = r2.index
            ip2 = p2.index            
            self.q[ir1,ir1,ir2] -= active(r1)*reaction.rateconst
            self.q[ir2,ir1,ir2] -= active(r2)*reaction.rateconst
            self.q[ip1,ir1,ir2] += active(p1)*reaction.rateconst
            self.q[ip2,ir1,ir2] += active(p2)*reaction.rateconst
                    
        elif(isinstance(reaction,ThreeBodyReaction)): # by default 2 reactants, 1 product
            r2 = species_dict[reaction.reactants[1]] # 2nd reactant
            ir2 = r2.index
            self.q[ir1,ir1,ir2] -= active(r1)*reaction.rateconst
            self.q[ir2,ir1,ir2] -= active(r2)*reaction.rateconst
            self.q[ip1,ir1,ir2] += active(p1)*reaction.rateconst
            
        else:
            sys.exit('Error, reaction type unrecognized!')

    def dpsidt(self,t,psi):
        """
        Inputs:
        t : time (not used, but needed for in-built python solver)
        psi : float64 array of length n_species
            chemical species number concentration [molecs/cm3].

        Output:
        dpsidt : float64 array of length n_species
            time derivative of chemical species number concentration [molecs/cm3/s].
        """

        # (psi_i)_t = A_ij*psi_j + q_ijk*psi_j*psi_k
        # (psi_i)_t = A_ij*psi_j + u_ij*psi_j
        # c.f. e.g. Brasseur & Jacob Ch. 6.
      
        AA = np.zeros((n_species,n_species))
        AA[:,:] = self.A[:,:]
        q_lay = self.q[:,:,:]        
        u = np.dot(q_lay,psi)
        dpsidt = np.dot(AA,psi) + np.dot(u,psi)

        return dpsidt

In [2]:
#### main code starts here  #####

# create list of elements
# (currently unused; could be used to do automated stoichiometry checks)
e_names=['C', 'H', 'O', 'N']

# create dictionary of species objects
species_dict={
    'O3':  ChemSpecies(0,'O3',  [0, 0, 3, 0],48.0,'active'), 
    'O2':  ChemSpecies(1,'O2',  [0, 0, 2, 0],32.0,'active'), 
    'O':   ChemSpecies(2,'O',   [0, 0, 1, 0],16.0,'active'),
    'NO':  ChemSpecies(3,'NO',  [0, 0, 1, 1],30.0,'active'),
    'NO2': ChemSpecies(4,'NO2', [0, 0, 2, 1],46.0,'active'),
    'HNO3':ChemSpecies(5,'HNO3',[0, 1, 3, 1],63.0,'active'),
    'OH':  ChemSpecies(6,'OH',  [0, 1, 1, 0],17.0,'active'),
    'H2O': ChemSpecies(7,'H2O', [0, 2, 1, 0],18.0,'active'),
    'NO3': ChemSpecies(8,'NO3', [0, 0, 3, 1],62.0,'active')
    }

# create dictionary of reaction objects

reactions={
    'R1': TwoBodyReaction('NO+O3=NO2+O2',lambda T: 1.40e-14*(T/298.)*exp(-1310/T)), #Jacob R9
    'R2': ThreeBodyReaction('O+O2=O3',lambda T: 6e-34*(T/298.)**(-2.3)), #Jacob R2
    'R3': PhotoReaction('NO2+hv=NO+O', 2.5e-3), #Jacob R10
    'R4': TwoBodyReaction('NO2+O=NO+O2',lambda T: 6.51e-12*(T/298.)*exp(120/T)), #Jacob R11
    'R5': ThreeBodyReaction('NO2+OH=HNO3', lambda T: 2.50e-30*(T/298.)**(-4.40)), #Jacob R12, k=0.81
    'R6': PhotoReaction('HNO3+hv=NO2+OH', lambda T: 9.33e15*exp(-24700/T)), #Jacob R16
    'R7': TwoBodyReaction('HNO3+OH=NO3+H20', lambda T: 1.52e-14*(T/298.)*exp(644/T)), #Jacob R17
    'R8': PhotoReaction('NO3+hv=NO2+O', 0.5) #Jacob R15; k obtained from Stark et al. 2007 (not available on NIST)
    }

# determine number of species and reactions
n_species = len(species_dict)
n_reactions = len(reactions)

# set atmospheric temperature [K]
T = 270.0

# define initial conditions for species concentrations
# numbers taken from Problem 10.2 of Jacob.
# @45km: n = 4.1e16, T = 270 K; @20km: n = 1.8e18, T = 200K
psi_tot = 4.1e16# total air number density [molec./cm3]
psi0 = np.zeros(n_species) # size of psi0 = number of species

psi0[0] = 2.0e11 # Jacob Fig 10-1
psi0[1] = 0.21*psi_tot
psi0[2] = 1.0e9 #3.0e7 @ 30 km, ~1.0e9 @ 45 km (rough agreement bwtn Ch. 10 values, 10.2 calcs, and chem solver)
psi0[3] = 1.0e6
psi0[4] = 1.0e6
psi0[5] = 1.0e3
psi0[6] = 4.1e6 #Li et al 2018, Nature (1.1e5 @ 10-12km) - rough agreement w/ Wennberg et al 1995; 1.4e6 @ 20km, 4.1e6 @ 40km
psi0[7] = 3e-6*psi_tot
psi0[8] = 1.0e3

# calculate non-photochemical reaction rates (could make this a loop)
reactions['R1'].calc_rate(T)
reactions['R2'].calc_rate(T,psi_tot)
reactions['R4'].calc_rate(T)
reactions['R5'].calc_rate(T,psi_tot)
reactions['R7'].calc_rate(T)

# create reaction network object and add all the reactions to it
reaction_network = ReactionNetwork(n_species)
reaction_network.add_reactions(reactions,species_dict)

# solve the system using in-built scipy solve_ivp function
time_tot = 1.0/S2DAY
sol = solve_ivp(reaction_network.dpsidt, [0, time_tot], psi0, method='BDF', dense_output=False)
psi = sol.y

# calculate error diagnostics
# (currently does not make use of elements tally)
initial_O = 7*psi0[0] + 6*psi0[1]  + 5*psi0[2]  + 4*psi0[3]  + 3*psi0[4]  + 2*psi0[5]  + 1*psi0[6]
err_in_O  = 7*psi[0] +  6*psi[1]   + 5*psi[2]   + 4*psi[3]   + 3*psi[4]   + 2*psi[5]  +  1*psi[6] - initial_O

# calculate expected O3 abundance analytically (c.f. Jacob eqn. 10.8)
C_O3_analytic = (1e-8*1e-33/(1e-2*8.e-12*exp(-2060/T)))**0.5*0.21*psi_tot**(3/2)

# display code results
fig = plt.figure(figsize = (20,15))
ax = fig.add_subplot(2,1,1)
ta=sol.t*S2DAY
plt.semilogy(ta,psi[0,:],ta,psi[1,:],ta,psi[2,:],ta,psi[3,:],ta,psi[4,:],ta,psi[5,:],ta,psi[6,:],ta,psi[7,:],ta,psi[8,:],ta,0*ta + C_O3_analytic)
plt.semilogy(ta,0*ta + C_O3_analytic,'k--')
plt.axis([0, time_tot*S2DAY, 1e-15*psi_tot, psi_tot])
plt.xlabel('t [days]')
plt.ylabel('[molecules/cm$^3$]')
plt.legend(['[O$_3$]', '[O$_2$]','[O]', '[NO]', '[NO$_2$]', '[HNO$_3$]', '[OH]', '[H$_2$O]', '[NO$_3$]'], shadow=True)

# display error in O conservation
ax2 = plt.subplot(2,1,2)
plt.plot(ta,1e2*err_in_O/initial_O)
plt.axis([0, time_tot*S2DAY, -1e-1, 1e-1])
plt.xlabel('t [days]')
plt.ylabel('% error in O')
plt.show()

TypeError: unsupported operand type(s) for *: 'bool' and 'function'

In [15]:
active = lambda species: species.behaviour=='active'
r1 = species_dict[reactions['R1'].reactants[0]]
active(r1)*3.0

3.0